In [2]:
import requests
url = "https://valencia.opendatasoft.com/api/records/1.0/search/"
params = {"dataset":"recursos-turisticos", "refine.objectid":85}
res = requests.get(url, params=params)
pdf_url = res.json()["records"][0]["fields"]["informacion_recurso"]


In [3]:
import requests
import io
import pdfplumber
res_pdf = requests.get(pdf_url)
with pdfplumber.open(io.BytesIO(res_pdf.content)) as pdf:
    texto = "\n".join(page.extract_text() for page in pdf.pages)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [3]:
texto

'Iglesia de Ntra. Sra. de los Ángeles\nFue fundada como ermita del gremio de pescadores a principios del siglo XIX en honor\na la Virgen. En 1791 el arquitecto José Fornés realiza el proyecto de la iglesia por\nencargo de la Comunidad de Marineros Matriculados, quien costeó las obras finalizadas\nen 1807. Situada en los límites que forman El Cabanyal con Cap de França, en la\nactualidad barrios de la ciudad de València, pero antiguamente poblados que formaban\nparte de Pueblo Nuevo del Mar. Es conocida como iglesia de El Cabanyal. Hay que\nmencionar que el nombre de Cabañal o «Cabanyal» procede de «cabañas» que era la\nconstrucción típica de esta parte de la costa.\nHacia finales del mismo siglo la iglesia había adquirido una nueva configuración en un\nestilo neoclásico muy mesurado que vagamente recuerda la actual construcción. La\ntitularidad como parroquia independiente la ostenta desde 1902, ya que con anterioridad\nla iglesia dependía de la de Santo Tomás de València a cuya demarc

In [5]:
import time
from gtts import gTTS

max_chars = 4000
chunks = [texto[i:i+max_chars] for i in range(0, len(texto), max_chars)]
total = len(chunks)

with open("recurso85.mp3", "wb") as f_out:
    for idx, chunk in enumerate(chunks, start=1):
        print(f"Procesando chunk {idx}/{total}…", end=" ", flush=True)
        t0 = time.time()
        try:
            tts = gTTS(text=chunk, lang="es")
            buf = io.BytesIO()
            tts.write_to_fp(buf)
            buf.seek(0)
            f_out.write(buf.read())
            print(f"Hecho en {time.time()-t0:.2f}s")
        except Exception as e:
            print("ERROR:", e)
            break

print("Proceso finalizado.")



Procesando chunk 1/2… Hecho en 49.25s
Procesando chunk 2/2… Hecho en 15.10s
Proceso finalizado.


In [7]:
import pyttsx3

engine = pyttsx3.init()
engine.setProperty("rate", 150)    # velocidad (palabras por minuto)
engine.setProperty("volume", 1.0)  # volumen 0.0 a 1.0

# De nuevo, si el texto es muy largo, trocéalo:
max_chars = 3000
chunks = [texto[i:i+max_chars] for i in range(0, len(texto), max_chars)]

for i, chunk in enumerate(chunks):
    # guardamos cada fragmento como un MP3/WAV separado
    filename = f"recurso85_part{i+1}.mp3"
    engine.save_to_file(chunk, filename)

# Ejecuta la cola de tareas y cierra el motor
engine.runAndWait()
print("Todos los fragmentos TTS guardados.")


Todos los fragmentos TTS guardados.


6. Itinerario y planificación
Creación de rutas combinadas: deja al usuario seleccionar varios puntos y genera la ruta óptima que pase por todos.

Estimación de tiempo: calcula duración total (bicicleta + paradas + tiempo de visita).